In [2]:
import pandas as pd
import numpy as np
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

In [3]:
fecha = '11_17_23'

In [30]:
DEU = pd.read_csv(f"./data/{fecha}/DEU_{fecha}.csv")
DEU_id = DEU['Contratante'].unique()
print(DEU.shape)
for name in DEU.select_dtypes(include=['object']):
    DEU[name] = DEU[name].str.replace("|", " ")
print("Hay que revisar DEU" if (DEU.isna().any(0).sum() > 2) else "Todo bien con DEU")
DEU.head(1)

(10909, 28)
Todo bien con DEU


,Clave Referencia,tipo distribuidor,Tipo Operación,Contratante,Paterno Deudor,Materno Contratante,Nombre Contratante,Tipo de Persona,RFC,Dirección,...,Crédito,Anexo,Monto Contrato,Frecuencia de Pago,Pago Periódico,Monto Pagare,Tasa,Fecha de nacimiento contratante,Sexo,CURP
0,1,CLUB,6,1,DIAZ,FLORES,MARIA DE LOS ANGELES,PF,DIFA770122,SAN BERNARDINO no ext.14 Cofradia de la Luz,...,1,N,62000,Quincenal,0,62000,NaN,1977-01-22,FEMENINO,DIFA770102MJCZLN08


In [31]:
DEU.columns

Index(['Clave Referencia', 'tipo distribuidor', 'Tipo Operación',
       'Contratante', 'Paterno Deudor', 'Materno Contratante',
       'Nombre Contratante', 'Tipo de Persona', 'RFC', 'Dirección', 'Ciudad',
       'CP', 'Estado', 'Email', 'Teléfono', 'Fecha Firma Contrato',
       'Fecha Vencimiento Contrato', 'Plazo', 'Crédito', 'Anexo',
       'Monto Contrato', 'Frecuencia de Pago', 'Pago Periódico',
       'Monto Pagare', 'Tasa', 'Fecha de nacimiento contratante', 'Sexo',
       'CURP'],
      dtype='object')

In [6]:
DEU.to_csv(f'./data/{fecha}/archivos_{fecha}/DEU_{fecha}.txt', sep ='|', index = False, header = False)

## MOVA

In [13]:
MOVA = pd.read_csv(f"./data/{fecha}/MOVA_{fecha}.csv")
MOVA['Fecha Vencimiento'] = MOVA['Fecha Vencimiento'].fillna("'2099-12-31'")
print("Hay que revisar MOVa" if (MOVA.isna().any(0).sum() > 0) else "Todo bien con MOVA")
MOVA_id = MOVA['Contrato']
print(MOVA.shape)
MOVA.head(1)

Todo bien con MOVA
(3473, 8)


,Contrato,Anexo,Número de documento,Fecha Vencimiento,Monto Capital,Monto intérés,Monto IVA,Tipo de Movimiento
0,81210,N,81210,2021-10-25,34000.0,8245.12,1319.22,FAC


In [9]:
MOVA = MOVA.loc[[mi in DEU_id for mi in MOVA_id]]

In [10]:
MOVA = MOVA.drop_duplicates('Contrato')

In [11]:
np.setdiff1d(MOVA['Contrato'], DEU_id)

array([], dtype=int64)

In [87]:
#MOVA['Número de documento'] = MOVA['Número de documento'].str.replace("'", "")

In [5]:
MOVA.columns

Index(['Contrato', 'Anexo', 'Número de documento', 'Fecha Vencimiento',
       'Monto Capital', 'Monto intérés', 'Monto IVA', 'Tipo de Movimiento'],
      dtype='object')

In [12]:
new_mova = []
for row in tqdm(MOVA.iterrows()):
    #new_mova = pd.concat([new_mova, row[1]], axis = 1)
    #new_mova.append(row[1])
    num = 16
    for i in range(1, num + 1):
        new_row = row[1].copy()
        new_row['Número de documento'] = str(new_row['Número de documento']) + "_"+str(i)
        new_row['Monto Capital'] = new_row['Monto Capital'] / num
        new_row['Monto intérés'] = new_row['Monto intérés'] / num
        new_row['Monto IVA'] = new_row['Monto IVA'] / num
        #new_mova = pd.concat([new_mova, new_row], axis = 1)
        new_mova.append(new_row)

2247it [00:02, 1037.12it/s]


In [13]:
MOVA = pd.DataFrame(new_mova)
MOVA.head()

,Contrato,Anexo,Número de documento,Fecha Vencimiento,Monto Capital,Monto intérés,Monto IVA,Tipo de Movimiento
0,81210,N,81210_1,2021-10-25,2125.0,515.32,82.45125,FAC
0,81210,N,81210_2,2021-10-25,2125.0,515.32,82.45125,FAC
0,81210,N,81210_3,2021-10-25,2125.0,515.32,82.45125,FAC
0,81210,N,81210_4,2021-10-25,2125.0,515.32,82.45125,FAC
0,81210,N,81210_5,2021-10-25,2125.0,515.32,82.45125,FAC


In [14]:
MOVA['Fecha Vencimiento'] = MOVA['Fecha Vencimiento'].str.replace("'", "")

In [14]:
MOVA

,Contrato,Anexo,Número de documento,Fecha Vencimiento,Monto Capital,Monto intérés,Monto IVA,Tipo de Movimiento
0,81210,N,81210,2021-10-25,34000.00,8245.12,1319.22,FAC
1,81274,N,81274,2021-10-19,54000.00,7373.32,1179.74,FAC
2,81352,N,81352,2021-10-27,33000.00,7867.22,1258.74,FAC
3,81425,N,81425,2021-11-08,32000.00,7630.24,1220.84,FAC
4,81466,N,81466,2021-10-21,35000.00,8341.18,1334.59,FAC
...,...,...,...,...,...,...,...,...
3468,169366,N,169366,2024-05-01,73018.84,18859.42,3017.74,FAC
3469,169384,N,169384,2024-03-06,16993.50,2329.80,372.70,FAC
3470,169388,N,169388,2023-12-27,34294.34,2111.79,337.87,FAC
3471,169405,N,169405,2024-03-07,43000.00,10848.10,1735.90,FAC


In [15]:
MOVA.shape

(35952, 8)

In [16]:
MOVA['Número de documento'].nunique()

35952

In [17]:
MOVA.to_csv(f'./data/{fecha}/archivos_{fecha}/MOVA_{fecha}.txt', sep ='|', index = False, header = False)

## MOVO

In [11]:
MOVO = pd.read_csv(f"./data/{fecha}/MOVO_{fecha}.csv")
print("Hay que revisar MOVO" if (MOVO.isna().any(0).sum() > 0) else "Todo bien con MOVO")
MOVO_id = MOVO['Contrato']
print(MOVO.shape)
MOVO.head(1)

Todo bien con MOVO
(292691, 11)


,Contrato,Anexo,Número de documento,Fecha Depósito,Fecha Aplicación,Monto de Principal,Monto de interés,Monto de IVA,Tipo de Movimiento,Cuenta,paymentId
0,80782,N,80782,2021-10-04,2021-10-04,-1199.75,-103.78,-16.6,1,BANCOMER,105234


In [12]:
MOVO.columns

Index(['Contrato', 'Anexo', 'Número de documento', 'Fecha Depósito',
       'Fecha Aplicación', 'Monto de Principal', 'Monto de interés',
       'Monto de IVA', 'Tipo de Movimiento', 'Cuenta', 'paymentId'],
      dtype='object')

In [19]:
sum([mi in DEU_id for mi in MOVO_id])

244162

In [20]:
MOVO = MOVO.loc[[mi in DEU_id for mi in MOVO_id]]

In [21]:
np.setdiff1d(MOVO['Contrato'], DEU_id)

array([], dtype=int64)

In [22]:
MOVO

,Contrato,Anexo,Número de documento,Fecha Depósito,Fecha Aplicación,Monto de Principal,Monto de interés,Monto de IVA,Tipo de Movimiento,Cuenta,paymentId
0,80782,N,80782,2021-10-04,2021-10-04,-1199.75,-103.78,-16.60,1,BANCOMER,105234
1,35680,N,35680,2021-10-04,2021-10-04,-1377.44,-278.22,-44.52,1,BANCOMER,105236
2,53379,N,53379,2021-10-04,2021-10-04,-2297.72,-592.16,-94.74,1,BANCOMER,105239
3,36018,N,36018,2021-10-04,2021-10-04,-4220.76,-1726.25,-276.20,1,BANCOMER,105243
4,54708,N,54708,2021-10-04,2021-10-04,-2520.98,-446.68,-71.47,1,BANCOMER,105245
...,...,...,...,...,...,...,...,...,...,...,...
292685,21882,N,21882,2023-09-30,2023-10-01,-6721.70,-2898.53,-463.77,1,BANCOMER,237303
292687,162872,N,143062,2023-09-30,2023-10-01,-438.00,0.00,0.00,1,BANCOMER,237301
292688,162872,N,114048,2023-09-30,2023-10-01,-438.00,0.00,0.00,1,BANCOMER,237301
292689,162872,N,162415,2023-09-30,2023-10-01,-438.00,0.00,0.00,1,BANCOMER,237301


In [24]:
MOVO.to_csv(f'./data/{fecha}/archivos_{fecha}/MOVO_{fecha}.txt', sep ='|', index = False, header = False)

## MOVIC

In [7]:
MOVIC = pd.read_csv(f"./data/{fecha}/MOVIC_{fecha}.csv")
print("Hay que revisar MOVIC" if (MOVIC.isna().any(0).sum() > 0) else "Todo bien con MOVIC")
MOVIC_id = MOVIC['Contrato']
print(MOVIC.shape)
MOVIC.head(1)

Todo bien con MOVIC
(292691, 10)


,Contrato,Anexo,Número de documento,Fecha Depósito,Fecha Aplicación,Monto de Seguro Asociado,Monto de Seguro Distribuidor,Tipo de Movimiento,Cuenta,paymentId
0,80782,N,80782,2021-10-04,2021-10-04,-50.0,-25.0,PAG,BANCOMER,105234


In [8]:
MOVIC.columns

Index(['Contrato', 'Anexo', 'Número de documento', 'Fecha Depósito',
       'Fecha Aplicación', 'Monto de Seguro Asociado',
       'Monto de Seguro Distribuidor', 'Tipo de Movimiento', 'Cuenta',
       'paymentId'],
      dtype='object')

In [26]:
sum([mi in DEU_id for mi in MOVIC_id])

244162

In [27]:
MOVIC = MOVIC.loc[[mi in DEU_id for mi in MOVIC_id]]

In [28]:
np.setdiff1d(MOVIC['Contrato'], DEU_id)

array([], dtype=int64)

In [29]:
MOVIC.to_csv(f'./data/{fecha}/archivos_{fecha}/MOVIC_{fecha}.txt', sep ='|', index = False, header = False)